In [1]:
from torch import optim
from lib.datasets import get_stock_price,sample_indices,train_test_split
from lib.aug import apply_augmentations,parse_augmentations
import torch
from torch import nn
import torch.nn.functional as F
from typing import List
from lib.utils import sample_indices


In [2]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 4,
}

In [3]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))
print("Before augmentation shape:",x_real_train.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_sig = apply_augmentations(x_real_train,sig_config["augmentations"])
    # Input dimension of encoder
    # We'll flat the tensor
    input_dim = x_aug_sig.shape[1]*x_aug_sig.shape[2]
print("After augmentation shape:",x_aug_sig.shape)
x_aug_sig = x_aug_sig.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


In [5]:
from models.cvae import CVAE,CAVE_train

# Model hyperparameters
lr = 1e-3  
batch_size = 128
epoch = 20001
latent_dim = 20  
hidden_dims = [156, 100, latent_dim]  

cvae = CVAE(
    x_aug_sig=x_aug_sig,
    x_original=x_real_train.to("cuda"),  
    epoch=epoch,
    batch_size=batch_size,
    hidden_dims=hidden_dims,
    latent_dim=latent_dim,
    device='cuda'
)
optimizer = torch.optim.Adam(cvae.parameters(), lr=lr)

# Train the model
CAVE_train(cvae, optimizer)

Epoch 0, Loss: 188601253888.0
Epoch 10, Loss: 94609326080.0
Epoch 20, Loss: 22843772928.0
Epoch 30, Loss: 3602438656.0
Epoch 40, Loss: 1706111872.0
Epoch 50, Loss: 579896832.0
Epoch 60, Loss: 255183184.0
Epoch 70, Loss: 164800656.0
Epoch 80, Loss: 127314248.0
Epoch 90, Loss: 102274240.0
Epoch 100, Loss: 115208496.0
Epoch 110, Loss: 98554120.0
Epoch 120, Loss: 96727376.0
Epoch 130, Loss: 90818448.0
Epoch 140, Loss: 93846192.0
Epoch 150, Loss: 96198640.0
Epoch 160, Loss: 111153024.0
Epoch 170, Loss: 100612288.0
Epoch 180, Loss: 94936096.0
Epoch 190, Loss: 102394832.0
Epoch 200, Loss: 104651952.0
Epoch 210, Loss: 100597072.0
Epoch 220, Loss: 100105936.0
Epoch 230, Loss: 96547520.0
Epoch 240, Loss: 90127264.0
Epoch 250, Loss: 92846256.0
Epoch 260, Loss: 97988048.0
Epoch 270, Loss: 93734200.0
Epoch 280, Loss: 93104728.0
Epoch 290, Loss: 103175200.0
Epoch 300, Loss: 93083640.0
Epoch 310, Loss: 103802336.0
Epoch 320, Loss: 94026784.0
Epoch 330, Loss: 95566224.0
Epoch 340, Loss: 89925008.0
Epo